<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preparing Data

In [2]:
dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/devdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/testdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/traindata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))

countPos = 0

def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][2]
    return 0

Data = []
Y = []

countPos = 0
index = -1
Data_test = []
Y_test = []

for i in range(0, len(dataset3)):
  val = checkPosNeg(dataset, dataset3[i][0])
  Y.append(val)
  try:
    fileName = "/content/drive/My Drive/MCA Dataset/303_P/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
  except Exception as e:
    print(e)

for i in range(0, len(dataset1)):
  val = checkPosNeg(dataset, dataset1[i][0])
  Y.append(val)
  try:
    fileName = "/content/drive/My Drive/MCA Dataset/302_P/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
  except Exception as e:
    print(e)

for i in range(0, len(dataset2)):
  Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
  try:
    fileName = "/content/drive/My Drive/MCA Dataset/300_P/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
  except Exception as e:
    print(e)

Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)

Data = []
Data_test = []
gc.collect()        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))


def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

def remove_StopWords(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

# def checkAcc(Y_pred, Y_test):
#     correct = 0
#     for i in range(len(Y_pred)):
#         if(Y_pred[i] == Y_test[i]):
#             correct+=1
    
#     return float(correct)/len(Y_pred)

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train


max_num_words = 20
max_num_sentence = 250

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words, 300))
max_length_sent = 0
sent = ""
for k in range(Data2.shape[0]):
    if(max_length_sent < len(Data2[k])):
      max_length_sent = len(Data2[k])
      sent = Data2[k]
    for i in range(min(max_num_sentence, len(Data2[k]))):
      try:
        sentence = Data2[k][i].split(" ")
      except:
        continue
      sentence = remove_StopWords(sentence)
      for j in range(min(max_num_words, len(sentence))):
        try:
          word = sentence[j]
          if(word[0] == '<'):
            if(word.find('>')!=-1):
              word = word[1:-1]
            else:
              word = word[1:]
          else:
            if(word.find('>')!=-1):
              word = word[0:-1]
          finalMatrix[k][i][j] = np.array(model[word])
        except Exception as e:
          continue
    	# try:
    	#   sentence = Data2[k][i].split(" ")
    	# except:
    	#   continue
    	# sentence = remove_StopWords(sentence)
    	# for j in range(min(max_num_words, len(sentence))):
    	# 	try:
    	# 	  word = sentence[j]
    	# 	  # print("Before", word)
    	# 	  if(word[0] == '<'):
    	# 	    if(word.find('>')!=-1):
    	# 	      word = word[1:-1]
    	# 	    else:
    	# 	      word = word[1:]
    	# 	  else:
    	# 	    if(word.find('>')!=-1):
    	# 	      word = word[0:-1]
    	# 	  finalMatrix[k][i][j] = np.array(model[word])
    	# 	except Exception as e:
    	# 		continue

#Test_data
max_length_sent = 0
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words, 300))
# print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    if(max_length_sent < len(Data2_test[k])):
      max_length_sent = len(Data2_test[k])
      sent = Data2_test[k]
    for i in range(min(max_num_sentence, len(Data2_test[k]))):
      try:
        sentence = Data2_test[k][i].split(" ")
      except:
        continue
      sentence = remove_StopWords(sentence)
      for j in range(min(max_num_words, len(sentence))):
        try:
          word = sentence[j]
          if(word[0] == '<'):
            if(word.find('>')!=-1):
              word = word[1:-1]
            else:
              word = word[1:]
          else:
            if(word.find('>')!=-1):
              word = word[0:-1]
          finalMatrix_test[k][i][j] = np.array(model[word])
        except Exception as e:
          continue

Data2 = []
Data2_test = []
model = []
stop_words = []
gc.collect()


finalMatrix, Y = upsample(finalMatrix,Y)
finalMatrix_test, Y_test = upsample(finalMatrix_test,Y_test)

<ipython-input-2-669aab2feb27>:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2 = np.array(Data2)
<ipython-input-2-669aab2feb27>:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2_test = np.array(Data2_test)


In [3]:
import keras
class CNN_Text:
  def __init__(self):
    # Initialising the CNN
    self.classifier = Sequential()
    # Step 1 - Convolution
    self.classifier.add(Conv2D(150, (1, 5), input_shape = (finalMatrix.shape[1], finalMatrix.shape[2], finalMatrix.shape[3]), activation = 'relu', data_format="channels_last"))
    # Step 2 - Pooling
    self.classifier.add(MaxPooling2D(pool_size = (1, 3)))
    # Adding a second convolutional layer
    self.classifier.add(Conv2D(75, (1, 3), activation = 'relu', data_format="channels_last"))
    self.classifier.add(MaxPooling2D(pool_size = (1, 2)))
    # Step 3 - Flattening
    self.classifier.add(Flatten())
    # Step 4 - Full connection
    self.classifier.add(Dense(units = 128, activation = 'relu'))
    self.classifier.add(Dense(units = 1, activation = 'sigmoid'))
    # Compiling the CNN
    lf = keras.losses.MeanSquaredLogarithmicError()
    self.classifier.compile(optimizer = 'adam', loss = lf, metrics = [keras.metrics.MeanSquaredError(), keras.metrics.AUC()])
    self.classifier = self.classifier

  def fitModel(self, X, Y, epoch = 10):
    self.classifier.fit(X, Y, epochs=epoch)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def modelEvaluate(self, X,Y):
    return self.classifier.evaluate(X,Y,verbose=1)
  
model = CNN_Text()
model.fitModel(finalMatrix, Y, 30)
# Y_Pred = model.predictModel(finalMatrix_test)

Epoch 1/30
2/2 [==============================] - 15s 1s/step - loss: 1.9405 - mean_squared_error: 50.2845 - auc: 0.6820
Epoch 2/30
2/2 [==============================] - 0s 100ms/step - loss: 1.5495 - mean_squared_error: 46.8851 - auc: 0.4184
Epoch 3/30
2/2 [==============================] - 0s 143ms/step - loss: 1.5471 - mean_squared_error: 46.8546 - auc: 0.5000
Epoch 4/30
2/2 [==============================] - 0s 119ms/step - loss: 1.5471 - mean_squared_error: 46.8545 - auc: 0.5000
Epoch 5/30
2/2 [==============================] - 0s 120ms/step - loss: 1.5471 - mean_squared_error: 46.8545 - auc: 0.5000
Epoch 6/30
2/2 [==============================] - 0s 106ms/step - loss: 1.5471 - mean_squared_error: 46.8545 - auc: 0.5000
Epoch 7/30
2/2 [==============================] - 0s 113ms/step - loss: 1.5471 - mean_squared_error: 46.8545 - auc: 0.5000
Epoch 8/30
2/2 [==============================] - 0s 125ms/step - loss: 1.5471 - mean_squared_error: 46.8545 - auc: 0.5000
Epoch 9/30
2/2 [==

In [4]:
scores = model.modelEvaluate(finalMatrix_test, Y_test)
print("Loss:", scores)

1/1 [==============================] - 0s 384ms/step - loss: 2.2363 - mean_squared_error: 102.9200 - auc: 0.5000
Loss: [2.236299514770508, 102.91999816894531, 0.5]
